In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

# ==========================
# Config
# ==========================
gt_dir = "/content/drive/MyDrive/data/images/gt_disp"          # ground truth disparity folder
pred_dir = "/content/drive/MyDrive/pre_disp_compitable"       # predicted disparity folder
out_dir = "./evaluation_out"  # output folder
os.makedirs(out_dir, exist_ok=True)
os.makedirs(f"{out_dir}/figs", exist_ok=True)
os.makedirs(f"{out_dir}/highlight_cases", exist_ok=True)

# ==========================
# Metrics functions
# ==========================
def compute_metrics(gt, pred, valid_mask, thresh=3.0):
    errors = np.abs(gt[valid_mask] - pred[valid_mask])
    mae = errors.mean()
    rmse = np.sqrt((errors**2).mean())
    bad_pix = (errors > thresh).sum() / valid_mask.sum()
    return mae, rmse, bad_pix

# ==========================
# Visualization helper
# ==========================
def visualize_case(name, left, gt, pred, error, valid_mask, save_path):
    plt.figure(figsize=(12, 4))

    # Left image
    plt.subplot(1, 4, 1)
    plt.imshow(left[..., ::-1])  # BGR -> RGB
    plt.title("Left Image")
    plt.axis("off")

    # GT disparity
    plt.subplot(1, 4, 2)
    plt.imshow(gt, cmap="jet")
    plt.title("GT Disparity")
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.axis("off")

    # Predicted disparity
    plt.subplot(1, 4, 3)
    plt.imshow(pred, cmap="jet")
    plt.title("Predicted Disparity")
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.axis("off")

    # Error map
    plt.subplot(1, 4, 4)
    err_map = np.zeros_like(gt)
    err_map[valid_mask] = error[valid_mask]   # ✅ fix
    plt.imshow(err_map, cmap="hot")
    plt.title("Error Map")
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.axis("off")

    plt.suptitle(f"Case: {name}", fontsize=14, fontweight="bold")
    plt.tight_layout()
    plt.savefig(save_path, dpi=200)
    plt.close()

# ==========================
# Main evaluation loop
# ==========================
results = []

for fname in tqdm(os.listdir(gt_dir)):
    if not fname.endswith(".png"):
        continue

    gt_path = os.path.join(gt_dir, fname)
    pred_path = os.path.join(pred_dir, fname)

    if not os.path.exists(pred_path):
        print(f"[!] Missing prediction for {fname}, skipping...")
        continue

    # load disparity maps
    gt = cv2.imread(gt_path, cv2.IMREAD_UNCHANGED).astype(np.float32)
    pred = cv2.imread(pred_path, cv2.IMREAD_UNCHANGED).astype(np.float32)

    # assume uint8 (0-255 disparity)
    gt = gt / 255.0
    pred = pred / 255.0

    # valid pixels (gt > 0)
    valid_mask = gt > 0

    mae, rmse, bad_pix = compute_metrics(gt, pred, valid_mask)
    results.append([fname, mae, rmse, bad_pix])

    # also visualize
    left_img_path = gt_path.replace("gt_disp", "left").replace("disp", "left")
    if os.path.exists(left_img_path):
        left_img = cv2.imread(left_img_path)
    else:
        left_img = np.zeros((gt.shape[0], gt.shape[1], 3), np.uint8)

    error = np.abs(gt - pred)
    save_path = f"{out_dir}/figs/{fname.replace('.png','')}_viz.png"
    visualize_case(fname, left_img, gt, pred, error, valid_mask, save_path)

# ==========================
# Save results table
# ==========================
df = pd.DataFrame(results, columns=["Image", "MAE", "RMSE", "BadPixelRate"])
df.loc["Average"] = df.mean(numeric_only=True)

csv_path = f"{out_dir}/metrics_results.csv"
df.to_csv(csv_path, index=False)
display(df.describe().transpose())

print(f"[✓] Evaluation finished. Results saved to: {csv_path}")
print(f"[✓] Visualizations saved in: {out_dir}/figs")

# ==========================
# Highlight best/worst cases
# ==========================
best_case = df.loc[df["RMSE"].idxmin()]
worst_case = df.loc[df["RMSE"].idxmax()]

print("\n=== Highlight Cases ===")
print("Best Case:", best_case.to_dict())
print("Worst Case:", worst_case.to_dict())

# Copy highlight visualizations
for case, label in [(best_case, "best"), (worst_case, "worst")]:
    case_name = case["Image"].replace(".png", "")
    src = f"{out_dir}/figs/{case_name}_viz.png"
    dst = f"{out_dir}/highlight_cases/{label}_case.png"
    if os.path.exists(src):
        os.system(f"cp '{src}' '{dst}'")

print(f"[✓] Highlight cases saved in: {out_dir}/highlight_cases")


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm


pred_dir = "/content/drive/MyDrive/pre_disp_compitable"
out_dir  = "/content/colored_depths"
os.makedirs(out_dir, exist_ok=True)


for fname in tqdm(sorted(os.listdir(pred_dir))):
    if not fname.endswith(".png"):
        continue


    depth = cv2.imread(os.path.join(pred_dir, fname), cv2.IMREAD_UNCHANGED)


    colored = cv2.applyColorMap(depth, cv2.COLORMAP_JET)


    cv2.imwrite(os.path.join(out_dir, fname), colored)




video_path = "/content/depth_video.mp4"
images = sorted([f for f in os.listdir(out_dir) if f.endswith(".png")])


frame = cv2.imread(os.path.join(out_dir, images[0]))
h, w, _ = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = 10
out = cv2.VideoWriter(video_path, fourcc, fps, (w, h))

for fname in images:
    img = cv2.imread(os.path.join(out_dir, fname))
    out.write(img)

out.release()
print(f"🎥 : {video_path}")
